# 对App review的筛选过滤

## 0. 引用

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import analyze_review
from analyze_review import red
import sample_size
import keywords_search
from pprint import pprint

## 1. 对UI-related和UI-unrelated review进行采样，并人工总结出false positive

## 目录

### 1.1 读取整体的数据集（在服务器上运行）

### 1.2 读取UI-related数据集 （在服务器上运行）

### 1.3 从整体的去除UI-related的数据，获得UI-unrelated的数据 （在服务器上运行）

### 1.4 整理好两部分的数据集后，固定随机种子，并采样（在服务器上运行）

### 1.5 读取采样的数据（可以在本地运行）

### 1.1 读取整体的数据集(在服务器上运行)

数据量非常大，需要在服务器上面跑

#### Free review的数量

In [2]:
# 测试加载时间
# %%timeit
# free = pd.read_csv("/home/qiuyuanchen/data/UI_data/not_used/Free_Apps_Reviews.csv.gz", compression="gzip")
# 3min 4s ± 1.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

3min 4s ± 1.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
free = pd.read_csv("/home/qiuyuanchen/data/UI_data/not_used/Free_Apps_Reviews.csv.gz", compression="gzip")
len(free)

75422963

#### Non-free review的数量

In [4]:
# %%timeit
# non_free = pd.read_csv("/home/qiuyuanchen/data/UI_data/not_used/Non_Free_Apps_Reviews.csv.gz", compression="gzip")
# 11.6 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

11.6 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
non_free = pd.read_csv("/home/qiuyuanchen/data/UI_data/not_used/Non_Free_Apps_Reviews.csv.gz", compression="gzip")
len(non_free)

3333370

#### 所有review的数量

In [6]:
# all=pd.concat([free, non_free])
# len(all)
# assert len(all) == (len(free) + len(non_free))

#### 直接读取所有的review，并check

In [8]:
check_all = pd.read_csv("/home/qiuyuanchen/data/UI_data/ALL_APP_REVIEW.csv.gz", compression="gzip")
assert len(all) == len(check_all)
len(check_all)

78756333

### 1.2 读取所有UI的数据

#### 所有UI的数量，以及free ui和non-free ui的数量

In [10]:
ui = pd.read_csv("/home/qiuyuanchen/data/UI_data/ALL_UI.csv")
red("UI review的数量")
print(len(ui))
red("Free UI review的数量")
free_ui = ui[ui["IS_FREE"] == 1]
print(len(free_ui))
red("Non free UI review的数量")
non_free_ui = ui[ui["IS_FREE"] == 0]
print(len(non_free_ui))

UI review的数量
3355145
Free UI review的数量
3035518
Non free UI review的数量
319627


### 1.3 从整体的去除UI-related的数据，获得UI-unrelated的数据 （需要在服务器上跑）

In [11]:
non_ui = all[~all["REVIEW_ID"].isin(ui["REVIEW_ID"])]

In [12]:
red("所有review的数量")
print(len(all))
red("UI review的数量")
print(len(ui))
red("UI-unrelated review的数量")

所有review的数量
78756333
UI review的数量
3355145
UI-unrelated review的数量
75401188


#### 直接读取处理好的UI-unrelated review 数据

In [27]:
check_non_ui = pd.read_csv("/home/qiuyuanchen/data/UI_data/ALL_NON_UI.csv.gz", compression="gzip")
assert len(check_non_ui) == len(non_ui)
print(len(check_non_ui))

75401188


#### 1.4 整理好两部分的数据集后，固定随机种子，并采样

In [13]:
red("UI review 采样的数量")
ui_size = sample_size.calculate_size(2.58, 0.03, len(ui))
print(ui_size)
red("UI-unrelated review 采样的数量")
non_ui_size = sample_size.calculate_size(2.58, 0.02, len(non_ui))
print(non_ui_size)

UI review 采样的数量
1848
UI-unrelated review 采样的数量
4160


In [35]:
RANDOM_SEED = 2333
sample_ui = ui.sample(ui_size, random_state=RANDOM_SEED).reset_index()
sample_non_ui = non_ui.sample(non_ui_size, random_state=RANDOM_SEED).reset_index()

In [82]:
pprint(sample_ui["REVIEW_TEXT"][0])
pprint(sample_non_ui["REVIEW_TEXT"][0])

("I've tried so many apps  this is the best for auto selection ( similar to "
 'magic wand function in Photoshop)  highly recommended')
'Very good'


#### 1.5 读取采样的数据（可以在本地运行）
输出成为html方便读

In [15]:
keywords = keywords_search.get_keywords("keywords.txt")

In [41]:
sample_ui = pd.read_csv("sample_ui.csv")
sample_non_ui = pd.read_csv("sample_non_ui.csv")
sample_ui['content'] = sample_ui["REVIEW_TITLE"].fillna("No title") + "\n" + sample_ui["REVIEW_TEXT"].fillna("No title")
sample_non_ui['content'] = sample_non_ui["REVIEW_TITLE"].fillna("No title") + "\n" + sample_non_ui["REVIEW_TEXT"].fillna("No title")

In [42]:
pprint(sample_ui['content'][0])
pprint(sample_non_ui['content'][0])

('No title\n'
 "I've tried so many apps  this is the best for auto selection ( similar to "
 'magic wand function in Photoshop)  highly recommended')
'No title\nVery good'


In [25]:
keywords_search.color_html(sample_ui['content'], keywords, "sample_ui.html")
keywords_search.color_html(sample_non_ui['content'], keywords, "sample_non_ui.html")

#### 分析采样的数据

In [80]:
red("在UI review的采样中，是UI review的数量以及比例")
label_ui = pd.read_csv("label_ui_false_positive.txt", header=None, squeeze=True, names=["is_ui"])
pprint(label_ui.value_counts())
pprint(label_ui.value_counts(normalize=True))
red("在UI-unrelated review的采样中，不是UI review的数量以及比例")
label_non_ui = pd.read_csv("label_non_ui_false_positive.txt", header=None, squeeze=True, names=["is_not_ui"])
pprint(label_non_ui.value_counts())
pprint(label_non_ui.value_counts(normalize=True))

在UI review的采样中，是UI review的数量以及比例
1    1812
0      36
Name: is_ui, dtype: int64
1    0.980519
0    0.019481
Name: is_ui, dtype: float64
在UI-unrelated review的采样中，不是UI review的数量以及比例
1    4133
0      27
Name: is_not_ui, dtype: int64
1    0.99351
0    0.00649
Name: is_not_ui, dtype: float64
